In [1]:
# Load data and set pandas options to display the full dataset
import re
import pandas as pd
import os

cur_dir = os.getcwd()
print(cur_dir)
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_colwidth', -1)  
restaurant_data = pd.read_csv(cur_dir +  '/Data/restaurants.tsv', sep='\t')
restaurant_data.head()

C:\Projekte\Uni\DMDB\dmdb


,id,name,address,city,phone,type
0,1,arnie morton's of chicago,435 s. la cienega blv.,los angeles,310/246-1501,american
1,2,arnie morton's of chicago,435 s. la cienega blvd.,los angeles,310-246-1501,steakhouses
2,3,art's delicatessen,12224 ventura blvd.,studio city,818/762-1221,american
3,4,art's deli,12224 ventura blvd.,studio city,818-762-1221,delis
4,5,hotel bel-air,701 stone canyon rd.,bel air,310/472-1211,californian


In [6]:
# drop the id column because we donÄt need it 
# don't drop id because it is important to check how good the results are
# restaurant_data = restaurant_data.drop(['id'], axis = 1)

In [2]:
# Get info about the data and datatypes of dthe DataFrame
restaurant_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 6 columns):
id         864 non-null int64
name       864 non-null object
address    864 non-null object
city       864 non-null object
phone      864 non-null object
type       863 non-null object
dtypes: int64(1), object(5)
memory usage: 40.6+ KB


In [3]:
# Remove special characters from phone number to detect duplicates
restaurant_data.phone = restaurant_data.phone.map(lambda x: re.sub(r'\W+', '', x))

In [9]:
# Check the unique values of city
restaurant_data.city.sort_values().unique()

array(['atlanta', 'bel air', 'beverly hills', 'brentwood', 'brooklyn',
       'burbank', 'century city', 'chinatown', 'college park',
       'culver city', 'decatur', 'duluth', 'encino', 'glendale',
       'hollywood', 'las vegas', 'long beach', 'los angeles', 'los feliz',
       'malibu', 'manhattan beach', 'mar vista', 'marietta',
       'marina del rey', 'monterey park', 'new york', 'northridge',
       'pacific palisades', 'pasadena', 'queens', 'rancho park',
       'redondo beach', 'roswell', 'san francisco', 'santa monica',
       'seal beach', 'sherman oaks', 'smyrna', 'st. boyle hts.',
       'st. hermosa beach', 'studio city', 'toluca lake', 'venice',
       'westlake village', 'westwood'], dtype=object)

In [10]:
# Map multiple occurencies of the same city in different writing
city_map = {'la': 'los angeles', 'new york city': 'new york', 'west la': 'los angeles', 'w. hollywood': 'hollywood'}
restaurant_data.city = restaurant_data.city.replace(city_map)
restaurant_data.city.unique()

array(['los angeles', 'studio city', 'bel air', 'sherman oaks',
       'santa monica', 'hollywood', 'malibu', 'beverly hills',
       'los feliz', 'chinatown', 'pasadena', 'new york', 'brooklyn',
       'las vegas', 'atlanta', 'san francisco', 'pacific palisades',
       'toluca lake', 'westlake village', 'northridge', 'mar vista',
       'venice', 'redondo beach', 'westwood', 'culver city', 'long beach',
       'century city', 'st. boyle hts.', 'rancho park',
       'st. hermosa beach', 'marina del rey', 'encino', 'monterey park',
       'burbank', 'seal beach', 'brentwood', 'manhattan beach',
       'glendale', 'queens', 'marietta', 'roswell', 'smyrna', 'duluth',
       'decatur', 'college park'], dtype=object)

In [12]:
# Check how many rows can be removed already
columns_no_id = restaurant_data.drop(['id'], axis=1).columns
restaurant_data[restaurant_data.duplicated(columns_no_id)].count()

id         10
name       10
address    10
city       10
phone      10
type       10
dtype: int64

name               854
address            854
city               854
phone              854
type               853
address_cleared    854
dtype: int64
name               854
address            854
city               854
phone              854
type               853
address_cleared    854
dtype: int64


In [13]:
# Remove unnecessary stuff fdrom the address column
print(len(restaurant_data.address.unique()))
restaurant_data['address_cleared'] = restaurant_data['address'].str.split(r' between| off| near| at').str[0]
len(restaurant_data.address_cleared.unique())

772


770

In [21]:
restaurant_data[['address_cleared', 'address', 'city']]

,address_cleared,address,city
0,435 s. la cienega blv.,435 s. la cienega blv.,los angeles
1,435 s. la cienega blvd.,435 s. la cienega blvd.,los angeles
2,12224 ventura blvd.,12224 ventura blvd.,studio city
3,12224 ventura blvd.,12224 ventura blvd.,studio city
4,701 stone canyon rd.,701 stone canyon rd.,bel air
5,701 stone canyon rd.,701 stone canyon rd.,bel air
6,14016 ventura blvd.,14016 ventura blvd.,sherman oaks
7,14016 ventura blvd.,14016 ventura blvd.,sherman oaks
8,624 s. la brea ave.,624 s. la brea ave.,los angeles
9,624 s. la brea ave.,624 s. la brea ave.,los angeles


In [27]:
print(restaurant_data.head())

   id                       name                  address         city  \
0  1   arnie morton's of chicago  435 s. la cienega blv.   los angeles   
1  2   arnie morton's of chicago  435 s. la cienega blvd.  los angeles   
2  3   art's delicatessen         12224 ventura blvd.      studio city   
3  4   art's deli                 12224 ventura blvd.      studio city   
4  5   hotel bel-air              701 stone canyon rd.     bel air       

        phone         type          address_cleared  
0  3102461501  american     435 s. la cienega blv.   
1  3102461501  steakhouses  435 s. la cienega blvd.  
2  8187621221  american     12224 ventura blvd.      
3  8187621221  delis        12224 ventura blvd.      
4  3104721211  californian  701 stone canyon rd.     


In [ ]:
# make mongodb connection 
from geopy.geocoders import Nominatim
import time
geolocator = Nominatim(user_agent="DMDB", timeout=100)
locations = []
for index, row in restaurant_data.iterrows():
    try: 
        location = geolocator.geocode(row.address_cleared + ' ' + row.city)
        time.sleep(2)
    except: 
        location = None
    if location is None :
        locations.append(None)
    else: 
        locations.append(location)
        print(location.address)
restaurant_data['location_new'] = locations